In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [3]:
sr = pd.read_csv("data/2017_all.csv")
schools = pd.read_csv("data/entities.csv")
#zips = pd.read_csv("data/zip_code_database.csv")

IOError: File data/2017_all.csv does not exist

In [ ]:
sanctuaries  = {'Alameda','Contra Costa','Los Angeles','Monterey','Napa','Riverside','Sacramento','San Bernardino','San Diego','San Francisco','San Mateo','Santa Clara','Santa Cruz','Sonoma'
}


our data frame sr,School Record, now contains contains all the information about student scores and populations. Our issue is that it also contains information about a number of other student groups we are not interested in, like disability status. the next issue we will want to deal with are the asterisks rows. from reading the survey methodology, we know these represent an ommitted score. scores are ommitted if they represent the average of fewer than 10 students. this concerns us because it means we dont have data for classes that contain fewer than 10 migrant students. there is an option to group students by school and not by grade will give us more schools with non-ommitted information for the migrant students. each of these entries has a school code we will join with the enties list that contains school and zip codes.

In [ ]:
#we remove any empty data as well as data for district summary. only non zero school codes map to actual schools.
mask = sr['Area 1 Percentage Below Standard'] != '*'
sr = sr[mask]
mask = sr['School Code'] > 0
sr = sr[mask]
sr['Mean Scale Score'] = pd.to_numeric(sr['Mean Scale Score'])
sr.shape

In [ ]:
#Selecting only the math scores to minimize language bias
mask = sr['Test Type']=='B'
sr = sr[mask]
mask = sr['Test Id'] == 2
#selecting the pool of all students at each school
mask = sr['Grade'] == 3
sr = sr[mask]

In [ ]:
#we will now segemnt to get two list of schools with the scores for the total and migrant population
all_mask = sr['Subgroup ID'] == 1
migrant_mask = sr['Subgroup ID'] == 28
school_all = sr[all_mask]
school_migrant = sr[migrant_mask]
school_all.iloc[3]

In [ ]:
school_all_scores = school_all[['School Code','Mean Scale Score']]
school_migrant_scores = school_migrant[['School Code','Mean Scale Score']]
school_all_scores = school_all_scores.rename(index=str, columns={"Mean Scale Score":"all score"})
school_migrant_scores = school_migrant_scores.rename(index=str, columns={"Mean Scale Score":"migrant score"})
print(np.mean(school_migrant_scores['migrant score']))

In [ ]:
#now we will join the two data frames using school code as the index
print(school_migrant_scores.shape)
school_all_scores = school_all_scores.drop_duplicates('School Code')
school_list = school_migrant_scores.merge(school_all_scores[['School Code','all score']],how='left',on='School Code')
print(school_list.shape)
#school_list

In [ ]:
_ = plt.hist(school_list['migrant score'],bins = 30,color ='red',alpha=.5,normed =True)
_ = plt.hist(school_list['all score'],bins =30,color = 'blue',alpha = .5,normed = True)
plt.show()
#all ready I think we can see the difference in distrobutions of the migrant and general population

In [ ]:
school_list = school_list.merge(schools,how='left',on='School Code')
school_list = school_list.rename(index=str, columns={"Zip Code_x": "zip"})
school_list.iloc[3]
#this gives location and district information for our schools
#we will be using this to test for sanctuary status

In [ ]:
#now we have each school with its zip code, total and migrant scores we will merge data about santuary counties
#zips.head()

In [ ]:
#ca_zips = zips[zips['state']=='CA']
#school_list = school_list.merge(ca_zips[['county','zip','latitude','longitude']],on='zip',how='left')
#school_list.iloc[3]

In [ ]:
#now we update this new sanctuary column based on whether the county is in our sanctuary list
school_list['sanctuary'] = np.empty(len(school_list))
for i in range(len(school_list)):
   # school_list['Filler'][1] = 1
    if school_list.ix[i,'County Name'] in sanctuaries:
        school_list.ix[i,'sanctuary'] = 1
    else:
        school_list.ix[i,'sanctuary'] = 0
    #print(school_list.ix[i,'sanctuary'])
school_list.head(2)

Fantastic, our data data now contains over 130 school with data about how migrant students perfrom relative to the general population and reference as to whether they are in a Sanctuary County.

In [ ]:
school_list.to_csv('data/third_math_AB.csv')

Important things to note about our data

This data does not include the scores of every student in california. Due to privacy concerns I can not access information about scores for groups of fewer than 10 students. This means that our data does not include data for schools with fewer than 10 migrant students per grade level. 
